In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../secretary')

import pandas as pd
from secretary import todo

In [16]:
td = todo.Todo()

tasks = [
    {
    "topic": "Shopping",
    "type": "action_item",
    "date": "2022-04-14 00:00:00",
    "requestor": "Jack",
    "actor": "Silvia",
    "summary": "Buy shoes",
    "details": "Siliva needs to buy some cool new kicks."
   },
  {
    "topic": "Grocery Shopping",
    "type": "action_item",
    "date": "2023-05-14 00:00:00",
    "requestor": "Jack",
    "actor": "Jack",
    "summary": "Buy eggs",
    "details": "I need to buy eggs"
  }
]

pd.DataFrame(tasks)

,topic,type,date,requestor,actor,summary,details
0,Shopping,action_item,2022-04-14 00:00:00,Jack,Silvia,Buy shoes,Siliva needs to buy some cool new kicks.
1,Grocery Shopping,action_item,2023-05-14 00:00:00,Jack,Jack,Buy eggs,I need to buy eggs


In [17]:
td.add_to_database(pd.DataFrame(tasks))
td.df

,topic,type,date,requestor,actor,summary,details,embedding
0,Shopping,action_item,2022-04-14 00:00:00,Jack,Silvia,Buy shoes,Siliva needs to buy some cool new kicks.,"[-0.0136561943218112, -0.016055431216955185, -..."
1,Grocery Shopping,action_item,2023-05-14 00:00:00,Jack,Jack,Buy eggs,I need to buy eggs,"[-0.04372985661029816, -0.002808217890560627, ..."


In [23]:

td.get_similar_records("2022-04-14 shoes")



0    0.410998
1    0.209267
Name: embedding, dtype: float64